In [1]:
# Cell 1

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("Titanic-Dataset.csv")
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# Cell 2

df = df.dropna()

target = "Survived"

# Encode categorical columns
for col in df.columns:
    if df[col].dtype == "object":
        df[col], _ = pd.factorize(df[col])

# Split features and label
X = df.drop(columns=[target]).values
y = df[target].values.reshape(-1, 1)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split (80:20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [3]:
# Cell 3

class LinearRegressionClassifier:
    def __init__(self, lr=0.01, epochs=2000):
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        m, n = X.shape
        self.W = np.zeros((n, 1))
        self.b = 0

        for _ in range(self.epochs):
            y_pred = X @ self.W + self.b
            error = y_pred - y

            dW = (1/m) * (X.T @ error)
            db = (1/m) * np.sum(error)

            self.W -= self.lr * dW
            self.b -= self.lr * db

    def predict(self, X):
        # Linear regression → output continuous values
        y_pred = X @ self.W + self.b
        
        # Convert to classification labels
        return (y_pred >= 0.5).astype(int)


In [4]:
# Cell 4

model = LinearRegressionClassifier(lr=0.01, epochs=3000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [5]:
# Cell 5

def macro_f1(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    classes = np.unique(y_true)

    f1_scores = []

    for c in classes:
        tp = np.sum((y_true == c) & (y_pred == c))
        fp = np.sum((y_pred == c) & (y_true != c))
        fn = np.sum((y_true == c) & (y_pred != c))

        precision = tp / (tp + fp + 1e-10)
        recall = tp / (tp + fn + 1e-10)
        f1 = 2 * precision * recall / (precision + recall + 1e-10)

        f1_scores.append(f1)

    return np.mean(f1_scores)

print("Macro F1 Score:", macro_f1(y_test, y_pred))


Macro F1 Score: 0.6792750196467727


In [6]:
# Cell 6

def accuracy(y_true, y_pred):
    return np.sum(y_true.flatten() == y_pred.flatten()) / len(y_true)

print("Accuracy:", accuracy(y_test, y_pred))


Accuracy: 0.7027027027027027
